# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
pd.options.mode.chained_assignment = None  # default='warn'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/clean_city_data.csv")
cities_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,12.06,69,81,4.92,AU,1662872718
1,Port Elizabeth,-33.9180,25.5701,14.62,88,17,2.24,ZA,1662872719
2,Rikitea,-23.1203,-134.9692,22.86,69,19,5.30,PF,1662872719
3,Albany,42.6001,-73.9662,19.70,88,100,0.77,US,1662872719
4,Thompson,55.7435,-97.8558,9.91,64,100,3.63,CA,1662872720
...,...,...,...,...,...,...,...,...,...
544,Garissa,-0.4536,39.6401,24.11,74,35,6.68,KE,1662873100
545,Brae,60.3964,-1.3530,10.02,83,45,2.49,GB,1662873101
546,Ouadda,8.0777,22.4007,19.60,98,91,1.28,CF,1662873102
547,Verkhoyansk,67.5447,133.3850,9.38,45,84,4.51,RU,1662873102


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Using the Lat and Lng as locations and Humidity as the weight
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"]

In [4]:
# Creating and adding heatmap
map = gmaps.figure()
heat = gmaps.heatmap_layer(locations, weights=humidity, point_radius=4, max_intensity=100, dissipating=False)
map.add_layer(heat)
plt.savefig("../Images/Humidity_Heatmap.png")
map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Creating a DataFrame with only ideal weather
temp_df = cities_df[(cities_df["Max Temp"] > 21) & (cities_df["Max Temp"] < 27)]
wind_df = temp_df[temp_df["Wind Speed"] < 4.5]
weather_df = wind_df[wind_df["Cloudiness"] == 0]
weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Carnarvon,-24.8667,113.6333,23.44,41,0,3.18,AU,1662872736
48,Ubari,26.5921,12.7805,22.02,37,0,1.54,LY,1662872747
59,Freeport,40.6576,-73.5832,23.31,76,0,3.09,US,1662872752
63,Saint George,37.1041,-113.5841,23.33,64,0,1.54,US,1662872755
67,Manises,39.4914,-0.4635,25.15,84,0,2.06,ES,1662872757
95,Moranbah,-22.0016,148.0466,26.82,24,0,2.08,AU,1662872777
97,Bethel,41.3712,-73.4140,21.06,79,0,0.45,US,1662872779
100,Phalaborwa,-23.9430,31.1411,21.34,25,0,2.11,ZA,1662872781
160,Beloha,-25.1667,45.0500,23.03,51,0,2.09,MG,1662872822
202,Rosetta,31.4044,30.4164,26.19,62,0,3.48,EG,1662872847


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Storing weather_df into hotel_df variable
hotel_df = weather_df
# Adding a "Hotel Name" column
hotel_df["Hotel Name"] = ""
# Setting parameters
parameters = {"radius" : 5000, "types" : "hotel", "keyword" : "hotels", "key" : g_key}

# Obtaining the latitude and longitude of each city and using it to find hotels
for x, row in hotel_df.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row["Latitude"]
    lng = row["Longitude"]
    parameters["location"] = str(lat) + ", " + str(lng)
    hotel = requests.get(url, params=parameters).json()
    try:
        hotel_df.loc[x, "Hotel Name"] = hotel["results"][0]["name"]
    except:
        hotel_df.loc[x, "Hotel Name"] = "N/A"
        print("No hotels found near " + str(row["City"]))
hotel_df

c:\Users\frank\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\Users\frank\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


No hotels found near Freeport
No hotels found near Beloha
No hotels found near Seminole


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,Carnarvon,-24.8667,113.6333,23.44,41,0,3.18,AU,1662872736,Hospitality Carnarvon
48,Ubari,26.5921,12.7805,22.02,37,0,1.54,LY,1662872747,فندق أوباري
59,Freeport,40.6576,-73.5832,23.31,76,0,3.09,US,1662872752,N/A
63,Saint George,37.1041,-113.5841,23.33,64,0,1.54,US,1662872755,Courtyard by Marriott St. George
67,Manises,39.4914,-0.4635,25.15,84,0,2.06,ES,1662872757,Hotel Port Feria
95,Moranbah,-22.0016,148.0466,26.82,24,0,2.08,AU,1662872777,Black Nugget Hotel Motel
97,Bethel,41.3712,-73.4140,21.06,79,0,0.45,US,1662872779,Hampton Inn Danbury
100,Phalaborwa,-23.9430,31.1411,21.34,25,0,2.11,ZA,1662872781,Bushveld Terrace Hotel on Kruger
160,Beloha,-25.1667,45.0500,23.03,51,0,2.09,MG,1662872822,N/A
202,Rosetta,31.4044,30.4164,26.19,62,0,3.48,EG,1662872847,فندق رشيد الدولي


In [13]:
# Drop rows where no hotels could be found
hotel_df = hotel_df[hotel_df["Hotel Name"]!="N/A"]
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,Carnarvon,-24.8667,113.6333,23.44,41,0,3.18,AU,1662872736,Hospitality Carnarvon
48,Ubari,26.5921,12.7805,22.02,37,0,1.54,LY,1662872747,فندق أوباري
63,Saint George,37.1041,-113.5841,23.33,64,0,1.54,US,1662872755,Courtyard by Marriott St. George
67,Manises,39.4914,-0.4635,25.15,84,0,2.06,ES,1662872757,Hotel Port Feria
95,Moranbah,-22.0016,148.0466,26.82,24,0,2.08,AU,1662872777,Black Nugget Hotel Motel
97,Bethel,41.3712,-73.4140,21.06,79,0,0.45,US,1662872779,Hampton Inn Danbury
100,Phalaborwa,-23.9430,31.1411,21.34,25,0,2.11,ZA,1662872781,Bushveld Terrace Hotel on Kruger
202,Rosetta,31.4044,30.4164,26.19,62,0,3.48,EG,1662872847,فندق رشيد الدولي
223,Old Saybrook,41.2918,-72.3762,21.69,83,0,2.06,US,1662872858,Water's Edge Resort & Spa
277,Shahr-e Kord,32.3256,50.8644,23.03,19,0,1.09,IR,1662872888,Parsian Hotel


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
map.add_layer(markers)
plt.savefig("../Images/Ideal_Weather_Hotels.png")
map

Figure(layout=FigureLayout(height='420px'))